In [1]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!ls

drive  sample_data


In [3]:
import os
os.chdir("/content/drive/My Drive")

In [4]:
!pip install gensim --upgrade
!pip install keras -- upgrade
!pip install pandas --upgrade


     |████████████████████████████████| 24.2MB 78.3MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
ERROR: Could not find a version that satisfies the requirement upgrade (from versions: none)
ERROR: No matching distribution found for upgrade
Requirement already up-to-date: pandas in /usr/local/lib/python3.6/dist-packages (1.0.5)


In [5]:
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten,Conv1D,MaxPooling1D,LSTM
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping


import nltk 
from nltk.corpus import stopwords 
from nltk.stem import SnowballStemmer

import gensim #Word2vec

import re
import numpy as np
from collections import Counter
import logging 
import time 
import pickle 
import itertools

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Using TensorFlow backend.


In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
COLUMNS = ["target", "ids", "date", "flag", "user", "text"]

DATASET_ENCODING  = "ISO-8859-1"

TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"
W2V_SIZE = 300
W2V_WINDOW = 7
W2V_EPOCH = 32
W2V_MIN_COUNT = 10

#SEQUENCE_LENGTH = 300
#EPOCHS = 8
#BATCH_SIZE = 1024

POSITIVE = "POSITIVE"
NEGATIVE = "NEGATIVE"
NEUTRAL = "NETUTRAL"
SENTIMENT_THRESHOLDS = (0.4,0.7)

In [8]:
os.chdir("sentiment_analysis_data")
!ls


training_1600000_processed_noemoticon.csv


In [9]:
data = pd.read_csv("training_1600000_processed_noemoticon.csv",encoding=DATASET_ENCODING,names =COLUMNS)

In [10]:
print("Dataset size",len(data))

Dataset size 1600000


In [11]:
data.head(5)

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [12]:
decode = {0:"NEGATIVE",2:"NEUTRAL",4:"POSITIVE"}
def decode_sentiment(label):
  return decode[int(label)]


In [13]:
data.target = data.target.apply(lambda x: decode_sentiment(x))

In [14]:
data.head(6)

,target,ids,date,flag,user,text
0,NEGATIVE,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,NEGATIVE,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,NEGATIVE,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,NEGATIVE,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,NEGATIVE,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
5,NEGATIVE,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [15]:
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")


In [16]:
def data_preprocessing(text,stem = False):
  text = re.sub(TEXT_CLEANING_RE, ' ',str(text).lower()).strip()
  tokens = []
  for token in text.split():
    if token not in stop_words:
      if stem:
        tokens.append(stemmer.stem(token))
      else:
        tokens.append(token)
  return " ".join(tokens)   

In [17]:
%%time
data.text = data.text.apply(lambda x:data_preprocessing(x))

CPU times: user 41.7 s, sys: 95.6 ms, total: 41.8 s
Wall time: 41.9 s


In [18]:
data_train,data_test = train_test_split(data,test_size = 0.2,random_state = 42)
print(len(data_train))
print(len(data_test))

1280000
320000


In [19]:
%%time
documents = [text.split() for text in data_train.text]


CPU times: user 2.3 s, sys: 297 ms, total: 2.6 s
Wall time: 2.6 s


In [20]:
data_train.text.head(5)

1374558    ya quot like palm pre touchstone charger ready...
1389115            felt earthquake afternoon seems epicenter
1137831                            ruffles shirts like likey
790714     pretty bad night crappy morning fml buttface d...
1117911                                      yeah clear view
Name: text, dtype: object

In [21]:
for i in range(0,5):
  print(documents[i])

['ya', 'quot', 'like', 'palm', 'pre', 'touchstone', 'charger', 'readynow', 'yes', 'sounds', 'good', 'beer', 'ready', 'prelaunch']
['felt', 'earthquake', 'afternoon', 'seems', 'epicenter']
['ruffles', 'shirts', 'like', 'likey']
['pretty', 'bad', 'night', 'crappy', 'morning', 'fml', 'buttface', 'didnt', 'say', 'could', 'go', 'work', 'today']
['yeah', 'clear', 'view']


In [22]:
w2v_model = gensim.models.word2vec.Word2Vec(size=W2V_SIZE,window=W2V_WINDOW,min_count=W2V_MIN_COUNT,workers = 8)

In [23]:
%%time
w2v_model.build_vocab(documents)

2020-07-01 13:32:22,898 : INFO : collecting all words and their counts
2020-07-01 13:32:22,901 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-07-01 13:32:22,922 : INFO : PROGRESS: at sentence #10000, processed 72565 words, keeping 14005 word types
2020-07-01 13:32:22,939 : INFO : PROGRESS: at sentence #20000, processed 144393 words, keeping 21587 word types
2020-07-01 13:32:22,957 : INFO : PROGRESS: at sentence #30000, processed 215826 words, keeping 27541 word types
2020-07-01 13:32:22,982 : INFO : PROGRESS: at sentence #40000, processed 288271 words, keeping 32764 word types
2020-07-01 13:32:23,001 : INFO : PROGRESS: at sentence #50000, processed 359772 words, keeping 37587 word types
2020-07-01 13:32:23,019 : INFO : PROGRESS: at sentence #60000, processed 431431 words, keeping 42198 word types
2020-07-01 13:32:23,041 : INFO : PROGRESS: at sentence #70000, processed 503103 words, keeping 46458 word types
2020-07-01 13:32:23,057 : INFO : PROGRESS: at s

CPU times: user 8.16 s, sys: 92.4 ms, total: 8.25 s
Wall time: 8.25 s


In [24]:
words = w2v_model.wv.vocab.keys()
vocab_size = len(words)
print("Vocab size:",vocab_size)

Vocab size: 30369


In [25]:
%%time
w2v_model.train(documents,total_examples = len(documents),epochs = W2V_EPOCH)

2020-07-01 13:32:34,430 : INFO : training model with 8 workers on 30369 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=7
2020-07-01 13:32:35,470 : INFO : EPOCH 1 - PROGRESS: at 5.22% examples, 416991 words/s, in_qsize 15, out_qsize 0
2020-07-01 13:32:36,479 : INFO : EPOCH 1 - PROGRESS: at 10.32% examples, 415930 words/s, in_qsize 16, out_qsize 1
2020-07-01 13:32:37,505 : INFO : EPOCH 1 - PROGRESS: at 15.95% examples, 427802 words/s, in_qsize 16, out_qsize 0
2020-07-01 13:32:38,520 : INFO : EPOCH 1 - PROGRESS: at 21.35% examples, 430496 words/s, in_qsize 12, out_qsize 3
2020-07-01 13:32:39,526 : INFO : EPOCH 1 - PROGRESS: at 26.99% examples, 436485 words/s, in_qsize 15, out_qsize 0
2020-07-01 13:32:40,546 : INFO : EPOCH 1 - PROGRESS: at 32.84% examples, 442338 words/s, in_qsize 15, out_qsize 0
2020-07-01 13:32:41,574 : INFO : EPOCH 1 - PROGRESS: at 38.35% examples, 442259 words/s, in_qsize 15, out_qsize 0
2020-07-01 13:32:42,635 : INFO : EPOCH 1 - PROGRESS: 

CPU times: user 18min 55s, sys: 3.98 s, total: 18min 59s
Wall time: 9min 45s


(263121332, 295270528)

In [26]:
w2v_model.most_similar("today")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
2020-07-01 13:42:20,156 : INFO : precomputing L2-norms of word weight vectors


[('tomorrow', 0.5097249746322632),
 ('day', 0.5043976306915283),
 ('afternoon', 0.5035260319709778),
 ('weekend', 0.49933087825775146),
 ('2day', 0.475668340921402),
 ('yesterday', 0.47545403242111206),
 ('morning', 0.47506076097488403),
 ('tonight', 0.4713689386844635),
 ('week', 0.43287205696105957),
 ('todays', 0.42525458335876465)]

In [27]:
%%time
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data_train.text)
vocab_size = len(tokenizer.word_index) + 1
print("Total words", vocab_size)

Total words 290419
CPU times: user 14.3 s, sys: 153 ms, total: 14.4 s
Wall time: 14.3 s


In [28]:
tokenizer.texts_to_sequences("how to learn")

[[1129], [], [181], [], [], [], [], [499], [475], [], [219], [134]]

In [29]:
x_train = pad_sequences(tokenizer.texts_to_sequences(data_train.text),maxlen=300)
x_test = pad_sequences(tokenizer.texts_to_sequences(data_test.text),maxlen=300)

In [30]:
print(x_train.shape)

(1280000, 300)


In [31]:
labels = data_train.target.unique().tolist()
labels

['POSITIVE', 'NEGATIVE']

In [32]:
labels.append(NEUTRAL)
labels

['POSITIVE', 'NEGATIVE', 'NETUTRAL']

In [33]:
encoder = LabelEncoder()
encoder.fit(data_train.target.tolist())

y_train = encoder.transform(data_train.target.tolist())
y_test = encoder.transform(data_test.target.tolist())

y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

print("y_train",y_train.shape)
print("y_test",y_test.shape)

y_train (1280000, 1)
y_test (320000, 1)


In [34]:
w2v_model.wv["good"]

array([-6.86772346e-01, -7.87019253e-01,  3.06356430e-01,  1.62609839e+00,
        7.18722701e-01, -7.76687682e-01, -1.79975820e+00,  5.30077934e-01,
       -1.58094430e+00, -2.19995707e-01, -8.44417885e-02, -8.97982240e-01,
        1.76936343e-01,  1.12525916e+00,  8.24672103e-01, -1.13308802e-03,
        8.29243958e-01, -1.17116439e+00, -1.65360406e-01,  1.38073349e+00,
       -9.90024507e-01,  5.06192565e-01,  4.70006056e-02,  6.84813499e-01,
        6.96642876e-01, -3.03975493e-01, -7.03662276e-01,  1.47860318e-01,
        1.03835428e+00,  4.07532692e-01,  6.80296898e-01, -1.14245951e+00,
       -3.49668682e-01,  7.69751549e-01, -4.67415601e-01, -1.28363776e+00,
        1.02275443e+00, -1.16138124e+00, -3.29158492e-02,  2.21407324e-01,
        2.94887245e-01,  6.07449770e-01,  3.93242650e-02,  7.28594482e-01,
       -8.35356340e-02,  1.32938230e+00, -3.40301208e-02,  1.48480749e+00,
       -3.91555876e-02,  1.66531086e-01, -1.45837843e-01, -4.00818795e-01,
       -7.42072701e-01, -

In [35]:
embedding_matrix = np.zeros((vocab_size,W2V_SIZE))
for word, i in tokenizer.word_index.items():
  if word in w2v_model.wv:
    embedding_matrix[i] = w2v_model.wv[word]
print(embedding_matrix.shape)

(290419, 300)


In [36]:
embedding_layer = Embedding(vocab_size,W2V_SIZE,weights=[embedding_matrix],
                            input_length=300,trainable= False)

In [37]:
model = Sequential()
model.add(embedding_layer)
model.add(Dropout(.5))
model.add(LSTM(100 , dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1,activation='sigmoid'))
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 300)          87125700  
_________________________________________________________________
dropout_1 (Dropout)          (None, 300, 300)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 87,286,201
Trainable params: 160,501
Non-trainable params: 87,125,700
_________________________________________________________________


In [38]:
model.compile(loss='binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [39]:
callbacks = [ ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
              EarlyStopping(monitor='val_accuracy', min_delta=1e-4, patience=5)]

In [40]:
%%time
history = model.fit(x_train,y_train,
                    batch_size=1024,
                    epochs=7,
                    validation_split=0.1,
                    verbose=1,
                    callbacks=callbacks)

Train on 1152000 samples, validate on 128000 samples
Epoch 1/7
1152000/1152000 [==============================] - 581s 505us/step - loss: 0.5147 - accuracy: 0.7427 - val_loss: 0.4708 - val_accuracy: 0.7743
Epoch 2/7
1152000/1152000 [==============================] - 590s 513us/step - loss: 0.4870 - accuracy: 0.7623 - val_loss: 0.4599 - val_accuracy: 0.7811
Epoch 3/7
1152000/1152000 [==============================] - 584s 507us/step - loss: 0.4804 - accuracy: 0.7662 - val_loss: 0.4582 - val_accuracy: 0.7834
Epoch 4/7
1152000/1152000 [==============================] - 588s 510us/step - loss: 0.4763 - accuracy: 0.7692 - val_loss: 0.4538 - val_accuracy: 0.7854
Epoch 5/7
1152000/1152000 [==============================] - 593s 515us/step - loss: 0.4733 - accuracy: 0.7710 - val_loss: 0.4523 - val_accuracy: 0.7861
Epoch 6/7
1152000/1152000 [==============================] - 606s 526us/step - loss: 0.4709 - accuracy: 0.7725 - val_loss: 0.4506 - val_accuracy: 0.7869
Epoch 7/7
1152000/1152000 [==

In [45]:
%%time
history = model.fit(x_train,y_train,
                    batch_size=1024,
                    epochs=1,
                    validation_split=0.1,
                    verbose=1,
                    callbacks=callbacks)

Train on 1152000 samples, validate on 128000 samples
Epoch 1/1
1152000/1152000 [==============================] - 577s 501us/step - loss: 0.4677 - accuracy: 0.7745 - val_loss: 0.4494 - val_accuracy: 0.7887
CPU times: user 13min 46s, sys: 1min 54s, total: 15min 41s
Wall time: 9min 37s


In [46]:
score = model.evaluate(x_test,y_test,batch_size=1024)
print()
print("ACCURACY:",score[1])
print("LOSS:",score[0])

320000/320000 [==============================] - 23s 73us/step

ACCURACY: 0.7886468768119812
LOSS: 0.44795261306762696


In [47]:
history.history

{'accuracy': [0.7744609],
 'loss': [0.46768004984325834],
 'lr': [0.001],
 'val_accuracy': [0.7886796593666077],
 'val_loss': [0.4493836431503296]}

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()


In [54]:
def predict(text):
    start_at = time.time()
    # Tokenize text
    x_test = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=300)
    # Predict
    score = model.predict([x_test])[0]
    # Decode sentiment
    if(score > 0.7):
      label = "POSITIVE"
    elif(score < 0.4):
      label = "NEGATIVE"
    else:
      label = "NEUTRAL"
    
    return {"label": label, "score": float(score),
       "elapsed_time": time.time()-start_at}  

In [61]:
predict("I hate this")

{'elapsed_time': 0.04774117469787598,
 'label': 'NEGATIVE',
 'score': 0.037131473422050476}